In [61]:
import pandas as pd
import requests
from io import StringIO

In [62]:
response = requests.get('https://dl.dropbox.com/s/06ni77wvrhukth3/scimagojr_full_2020-03-31.csv')
content = StringIO(response.content.decode("utf8"))
df = pd.read_table(content, sep=";")

df.drop(['Sourceid', 'Issn', 'SJR',
 'H index', 'Total Docs. (2018)', 'Total Docs. (3years)', 'Total Refs.',
 'Total Cites (3years)', 'Citable Docs. (3years)', 'Cites / Doc. (2years)',
 'Ref. / Doc.', 'Country', 'Publisher', 'Coverage', 'Categories'], axis=1, inplace=True)

In [63]:
print(df.columns.values)
df.sample(10)

['Unnamed: 0' 'Rank' 'Title' 'Type' 'SJR Quartile' 'Area']


,Unnamed: 0,Rank,Title,Type,SJR Quartile,Area
64584,34,35,Ultrasound,journal,Q3,Radiological and Ultrasound Technology
57939,55,56,Knee,journal,Q1,Orthopedics and Sports Medicine
46722,212,213,Society of Petroleum Engineers - SPE Russian P...,conference and proceedings,-,Management of Technology and Innovation
1933,204,205,Journal of Poultry Science,journal,Q3,Animal Science and Zoology
13401,14,15,WWW 2015 - Proceedings of the 24th Internation...,conference and proceedings,-,Computer Graphics and Computer-Aided Design
54946,4,5,Ecosystem Services,journal,Q1,Nature and Landscape Conservation
15893,2172,2173,Education Division 2015 - Core Programming Are...,conference and proceedings,-,Computer Networks and Communications
16330,413,414,Entertainment Computing,journal,Q3,Computer Science Applications
66251,146,147,Kerntechnik,journal,Q3,"Safety, Risk, Reliability and Quality"
31494,11,12,Emerging Infectious Diseases,journal,Q1,Epidemiology


In [64]:
df_cs = df.loc[df['Area'].str.contains('computer science', case = False, regex = False) & (df['SJR Quartile'].str.contains('Q1|Q2'))]
# df_ling = df.loc[df['Area'].str.contains('linguistics', case = False, regex = False) & (df['SJR Quartile'].str.contains('Q1|Q2'))]
df_ling = df.loc[df['Area'].str.contains('linguistics', case = False, regex = False)]

In [65]:
s1 = pd.merge(df_cs, df_ling, how='inner', on=['Title'])

In [66]:
s1

,Unnamed: 0_x,Rank_x,Title,Type_x,SJR Quartile_x,Area_x,Unnamed: 0_y,Rank_y,Type_y,SJR Quartile_y,Area_y
0,53,54,Computer Assisted Language Learning,journal,Q1,Computer Science Applications,22,23,journal,Q1,Language and Linguistics
1,53,54,Computer Assisted Language Learning,journal,Q1,Computer Science Applications,26,27,journal,Q1,Linguistics and Language
2,68,69,Language Learning and Technology,journal,Q1,Computer Science Applications,28,29,journal,Q1,Language and Linguistics
3,68,69,Language Learning and Technology,journal,Q1,Computer Science Applications,31,32,journal,Q1,Linguistics and Language
4,80,81,ReCALL,journal,Q1,Computer Science Applications,34,35,journal,Q1,Language and Linguistics
5,80,81,ReCALL,journal,Q1,Computer Science Applications,40,41,journal,Q1,Linguistics and Language
6,210,211,Synthesis Lectures on Human Language Technologies,book series,Q2,Computer Science Applications,119,120,book series,Q1,Linguistics and Language
7,263,264,Speech Communication,journal,Q2,Computer Science Applications,127,128,journal,Q1,Language and Linguistics
8,263,264,Speech Communication,journal,Q2,Computer Science Applications,139,140,journal,Q1,Linguistics and Language
9,296,297,Computational Linguistics,journal,Q2,Computer Science Applications,137,138,journal,Q1,Language and Linguistics
